## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
%matplotlib inline

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
0,0,Hobart,-42.8794,147.3294,58.26,70,100,3.00,overcast clouds,AU,2022-07-23 02:35:32
1,1,Mataura,-46.1927,168.8643,45.48,65,4,2.84,clear sky,NZ,2022-07-23 02:40:21
2,2,Tasiilaq,65.6145,-37.6368,35.76,80,44,7.45,scattered clouds,GL,2022-07-23 02:35:46
3,3,Kula,38.5473,28.6498,61.32,50,0,4.94,clear sky,TR,2022-07-23 02:40:22
4,4,Avera,33.1940,-82.5271,80.65,67,99,5.57,overcast clouds,US,2022-07-23 02:40:22


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                248
City                   248
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Weather Description    248
Country                248
Date                   248
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [17]:
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
4,4,Avera,33.1940,-82.5271,80.65,67,99,5.57,overcast clouds,US,2022-07-23 02:40:22
5,5,Bethel,41.3712,-73.4140,77.68,90,0,0.00,clear sky,US,2022-07-23 02:35:30
11,11,Ca Mau,9.1769,105.1500,87.31,66,74,1.21,broken clouds,VN,2022-07-23 02:38:26
19,19,Teahupoo,-17.8333,-149.2667,75.16,78,34,11.65,scattered clouds,PF,2022-07-23 02:40:28
27,27,Avarua,-21.2078,-159.7750,78.85,78,40,16.11,scattered clouds,CK,2022-07-23 02:40:31
...,...,...,...,...,...,...,...,...,...,...,...
705,705,Bugko,12.5356,124.7923,87.30,66,100,5.12,overcast clouds,PH,2022-07-23 03:00:01
707,707,Dingle,10.9995,122.6711,86.68,71,92,4.41,overcast clouds,PH,2022-07-23 03:00:02
709,709,Pangai,-19.8000,-174.3500,75.63,75,98,18.37,overcast clouds,TO,2022-07-23 03:00:04
711,711,Machhlishahr,25.6833,82.4167,88.20,66,100,16.51,overcast clouds,IN,2022-07-23 03:00:05


In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Avera,US,80.65,overcast clouds,33.1940,-82.5271,
5,Bethel,US,77.68,clear sky,41.3712,-73.4140,
11,Ca Mau,VN,87.31,broken clouds,9.1769,105.1500,
19,Teahupoo,PF,75.16,scattered clouds,-17.8333,-149.2667,
27,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
33,Atuona,PF,77.43,few clouds,-9.8000,-139.0333,
35,Kadi,IN,82.26,overcast clouds,23.3000,72.3333,
37,Yuancheng,CN,75.07,clear sky,36.6520,107.7896,
39,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,
41,Albany,US,77.41,broken clouds,42.6001,-73.9662,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))